In [ ]:
import re
import math
from itertools import chain
from typing import Tuple, List

part 1

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:10]

In [ ]:
seeds = [int(n) for n in re.findall(r"(\d+)", lines[0])]
seeds[:2]

In [ ]:
# parse input file mappings
mappings_list = []
current_mapping_name = None
current_mapping_list = None
for l in lines[1:]:
    if not l:
        if current_mapping_name:
            mappings_list.append((current_mapping_name, current_mapping_list))
        else:
            continue
    elif l[0] not in "0123456789":
        # start new mapping
        current_mapping_name = l.split(" ")[0]
        current_mapping_list = []
    else:
        assert current_mapping_list is not None
        current_mapping_list.append([int(n) for n in re.findall(r"(\d+)", l)])
mappings_list.append((current_mapping_name, current_mapping_list))

In [ ]:
# idea is to get (start,end,shift) for each mapping
# so for every value between start and end, the mapping is simply value+shift
def get_category_map(category):
    category_map_triples = []
    for mapping in category:
        dest_range_start = mapping[0]
        source_range_start = mapping[1]
        mapping_len = mapping[2]

        category_map_triples.append(
            (
                source_range_start,
                source_range_start + mapping_len,
                dest_range_start - source_range_start,
            )
        )

    return sorted(category_map_triples, key=lambda x: x[0])


mappings_list_res = [(n, get_category_map(m)) for n, m in mappings_list]
# mappings_list_res

In [ ]:
def resolve_mapping(num, mapping):
    mapping = mapping[1]
    for m in mapping:
        if num < m[1] and num >= m[0]:
            return num + m[2]
        elif num < m[0]:
            break
        else:
            continue
    return num


# test cases for example input
print(resolve_mapping(10, mappings_list_res[0]))
print(resolve_mapping(60, mappings_list_res[0]))
print(resolve_mapping(99, mappings_list_res[0]))

In [ ]:
def resolve_location(seed):
    for cat_map in mappings_list_res:
        # print(cat_map[0])
        seed_new = resolve_mapping(seed, cat_map)
        # print(f"{soil} -> {soil_new}")
        seed = seed_new
    return seed

In [ ]:
min_loc = math.inf
for seed in seeds:
    location = resolve_location(seed)
    # print(soil)
    min_loc = min(location, min_loc)
min_loc

part 2

In [ ]:
# Mapping all values is too slow, so instead of that entire ranges are mapped into one or more mapped ranges


def map_range(
    r: Tuple[int, int], seed_loc_mapping: Tuple[str, List[Tuple[int, int, int]]]
) -> List[Tuple[int, int]]:
    # target is a list of mapped ranges
    res_ranges = []
    # copy the actual mapping part, so it can be manipulated
    mapping = seed_loc_mapping[1].copy()
    range_start = r[0]
    range_end = r[1]

    while len(mapping) > 0:
        # take first mapping, they are sorted in increasing intervals
        curr_mapping = mapping.pop(0)
        # print(f"Curr mapping: {curr_mapping}")

        # current mapping before range, so drop it
        if range_start > curr_mapping[1]:
            continue

        # check if a part of the source range happens before the current mapping
        if range_start < curr_mapping[0]:
            # if so, add it without shifting and move range_start accordingly
            res_ranges.append((range_start, curr_mapping[0], 0))
            range_start = curr_mapping[0]

        # do the part that falls into the current mapping
        if range_start >= curr_mapping[0] and range_end <= curr_mapping[1]:
            # and break if that covers the full source already
            res_ranges.append((range_start, range_end, curr_mapping[2]))
            range_start = range_end
            break
        elif (range_start >= curr_mapping[0]) and (range_end > curr_mapping[1]):
            # or occupy the complete mapping range
            res_ranges.append((range_start, curr_mapping[1], curr_mapping[2]))
            # and continue after it
            range_start = curr_mapping[1]

    # check if room is open at the end, and add without shifting
    if range_start < range_end:
        res_ranges.append((range_start, range_end, 0))

    # return mapped ranges with shift already applied
    return [(r[0] + r[2], r[1] + r[2]) for r in res_ranges]


# Test cases for example input
# map_range((79, 93), mappings_list_res[0])
map_range((68, 81), mappings_list_res[5])

In [ ]:
# map initial range tuple iteratively through all mapping stages
# if a mapping has multiple ranges as result, map all of them in the next step
def full_range_mapping(
    rng: Tuple[int, int], mappings: List[Tuple[str, List[Tuple[int, int, int]]]]
):
    r_list = [rng]
    for m in mappings:
        r_list = [map_range(r, m) for r in r_list]
        # flatten
        r_list = list(chain.from_iterable(r_list))
    return r_list


# Test cases for example input
full_range_mapping((79, 79 + 14), mappings_list_res)

In [ ]:
min_loc = math.inf
# Map all ranges for part 2, sort ranges by their start values
# very first value is min possible value for a range
for i in range(0, len(seeds), 2):
    start = seeds[i]
    r = seeds[i + 1]
    print(f"Start {start}, len {r}")

    rl = full_range_mapping((start, start + r), mappings_list_res)
    rl = sorted(rl, key=lambda x: x[0])

    rl_m = rl[0][0]
    min_loc = min(rl_m, min_loc)

min_loc